- 코사인 유사도
    - 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도를 의미
    - 코사인 유사도는 -1이상 1이하의 값을 가짐
    - 값이 1에 가까울수록 유사도가 높다고 판단
    
  ![Cosine Similarity Shape](./CosineSimilarity_shape.png)

  ![Cosine Similarity Equation](./CosineSimilarity_equation.png)

In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print('문서 1과 문서2의 유사도 :', cos_sim(doc1, doc2))
print('문서 1과 문서3의 유사도 :', cos_sim(doc1, doc3))
print('문서 2과 문서3의 유사도 :', cos_sim(doc2, doc3))

문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 1.0000000000000002


- 특징
    - 문서 1과 2의 유사도와 문서 1과 2의 유사도 같음
    - 문서 2과 3의 유사도는 1임
        - 1은 두 벡터의 방향이 완전히 동일하고, 유사도의 값이 최대임
        - 문서 3은 문서 2에서 모든 단어의 빈도수가 1씩 증가함. 즉, 한 문서내의 모든 단어의 빈도수가 동일하게 증가하는 경우의 유사도는 1이 나옴
        - 코사인 유사도는 유사도를 구할 때, 방향에 초점을 맞추므로, 문서의 길이가 다른 상황에서 비교적 공정한 비교가 가능해짐
        - 두 문서는 동일한 주제의 문서일 가능성이 높다는 것을 시사한다.

유사도를 이용한 추천 시스템 구현하기

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('./data/movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
data = data.head(20000)

In [5]:
#tf-idf 계산할 때, null값이 들어 있으면 에러 발생
print('overview 열의 결측값의 수:', data['overview'].isnull().sum())

overview 열의 결측값의 수: 135


In [6]:
#결측값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('')

In [8]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기 (shape) :', tfidf_matrix.shape)

TF-IDF 행렬의 크기 (shape) : (20000, 47487)


In [13]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과: ', cosine_sim.shape)

코사인 유사도 연산 결과:  (20000, 20000)


In [14]:
title_to_index = dict(zip(data['title'], data.index))

idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [25]:
#영화의 제목을 입력하면 코사인 유사도를 통해 가장 overview가 유사한 10개의 영화 찾기
def get_recommendations(title, cosine_sim=cosine_sim):
    #영화의 타이틀에 대응되는 인덱스 받아오기
    idx = title_to_index[title]
    #해당 영화와 모든 영화와의 유사도 가져오기
    sim_scores = list(enumerate(cosine_sim[idx]))
    #유사도에 따라 영화들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    #가장 유사한 10개의 영화 받아오기
    sim_socres = sim_scores[1:11]
    #가장 유사한 10개의 영화 인덱스 얻기
    movie_indices = [idx[0] for idx in sim_scores]
    #가장 유사한 10개의 영화 제목 리턴
    return data['title'].iloc[movie_indices]

In [26]:
#다크 나이트 라이즈와 overview가 유사한 영화 찾기
get_recommendations('The Dark Knight Rises')

18252                                The Dark Knight Rises
12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
                               ...                        
19992                          How to Make Love to a Woman
19994                               Violeta Went to Heaven
19996                                           Versailles
19998    Lotte Reiniger: Homage to the Inventor of the ...
19999    RKO Production 601: The Making of 'Kong, the E...
Name: title, Length: 20000, dtype: object